<a href="https://colab.research.google.com/github/slvg01/01_becode_follow_the_white_rabbit/blob/main/GenAI_Sylvain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wandb
!pip install --upgrade --quiet diffusers accelerate

import wandb
import numpy as np
from PIL import Image
import json
from diffusers import DiffusionPipeline
import torch
from copy import deepcopy
import gc




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 62.0 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:

torch.cuda.empty_cache()
gc.collect()
refiner=None

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")

base_vae=deepcopy(base.vae)
base_text_encoder_2=deepcopy(base.text_encoder_2)

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40
high_noise_frac = 0.8

prompt = '''A family with parents and 2 teenagers, all calling each others
with their phone. they look happy and vivid'''

# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images

torch.cuda.empty_cache()
gc.collect()
base=None

refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base_text_encoder_2,
    vae=base_vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

model_index.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/4.52G [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
wandb.login(key='904889813ecd4cce420d8cf3a85a5b7157c9ba35')

# Change this with the path of the json file containing your parameters
#with open('parameters.json', 'r') as file:
    #config = json.load(file)
# Initialize W&B run
wandb.init(project="image-generation", name="experiment-2", config={'model':'stabilityai/stable-diffusion-xl-base-1.0', 'torch_dtype':'torch.float16', 'variant':'fp16', 'use_safetensors':'True' })

# Example: Generate and save images during training (dummy example)
#prompt = # your prompt object here
#image = # your image object here from the model
image_path = f"generated_image_epoch.png" # Change the name as you like, you can have an image for each epoch or only the last one
image.save(image_path) # to save the image to the path and name specified above

# Log training metrics and the generated image
wandb.log({
    "prompt": prompt, # Log the prompt
    #"epoch": epoch, # define this in your training loop
    #"loss": loss,  # Dummy loss value, replace with your actual loss or remove
    "generated_image": wandb.Image(image_path) # Log the image
})

# Finish the W&B run
wandb.finish()


wandb: Currently logged in as: sylvain-legay (becode). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


prompt,A family with parent...
